In [1]:
# EOMs
%reset
#import libraries
import sympy as sym
import numpy as np
import pickle as pickle

sym.init_printing()
from IPython.display import display #for pretty printing

# Symbols --------------------------------------------------
links = ['body','UL','UR','footL','footR']

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#Parameters
masses = {}
lengths = {}
inertias = {}
th = {}
dth = {}
ddth = {}
r_prismatic = {}
dr_prismatic = {}
ddr_prismatic = {}

g = sym.symbols('g')

#gen coordinates
x_body,y_body,th_body = sym.symbols(['x_body','y_body','theta_{0}'.format('body')])
dx_body,dy_body,dth_body = sym.symbols(['\dot{\\x}_{body}','\dot{\\y}_{body}','\dot{\\theta}_{body}'])
ddx_body,ddy_body,ddth_body = sym.symbols(['\ddot{\\x}_{body}','\ddot{\\y}_{body}','\ddot{\\theta}_{body}'])

for link in links:
    masses.update({(link): sym.symbols('m_{0}'.format(link))})
    lengths.update({(link): sym.symbols('l_{0}'.format(link))})
    inertias.update({(link): sym.symbols('In_{0}'.format(link))})
    
    if 'body' in link:
        pass
    if 'foot' in link:
        r_prismatic.update({(link): sym.symbols('{{r\_prismatic}}_{{{0}}}'.format(link))})
        dr_prismatic.update({(link): sym.symbols('\dot{{\\r\_prismatic}}_{{{0}}}'.format(link))})
        ddr_prismatic.update({(link): sym.symbols('\ddot{{\\r\_prismatic}}_{{{0}}}'.format(link))})
    else:
        th.update({(link): sym.symbols('theta_{0}'.format(link))}) 
        dth.update({(link): sym.symbols('\dot{{\\theta}}_{{{0}}}'.format(link))})
        ddth.update({(link): sym.symbols('\ddot{{\\theta}}_{{{0}}}'.format(link))})

In [3]:
q = sym.Matrix([x_body]+[y_body]+[th[link] for link in links if 'foot' not in link]+[r_prismatic[link] for link in links if 'foot' in link])
dq = sym.Matrix([dx_body]+[dy_body]+[dth[link] for link in links if 'foot' not in link]+[dr_prismatic[link] for link in links if 'foot' in link])
ddq = sym.Matrix([ddx_body]+[ddy_body]+[ddth[link] for link in links if 'foot' not in link]+[ddr_prismatic[link] for link in links if 'foot' in link])

In [4]:
q

⎡        x_body        ⎤
⎢                      ⎥
⎢        y_body        ⎥
⎢                      ⎥
⎢        θ_body        ⎥
⎢                      ⎥
⎢         θ_UL         ⎥
⎢                      ⎥
⎢         θ_UR         ⎥
⎢                      ⎥
⎢{r\_prismatic}_{footL}⎥
⎢                      ⎥
⎣{r\_prismatic}_{footR}⎦

In [5]:
dq

⎡      \dot{\x}_{body}      ⎤
⎢                           ⎥
⎢      \dot{\y}_{body}      ⎥
⎢                           ⎥
⎢    \dot{\theta}_{body}    ⎥
⎢                           ⎥
⎢     \dot{\theta}_{UL}     ⎥
⎢                           ⎥
⎢     \dot{\theta}_{UR}     ⎥
⎢                           ⎥
⎢\dot{\r\_prismatic}_{footL}⎥
⎢                           ⎥
⎣\dot{\r\_prismatic}_{footR}⎦

In [6]:
ddq

⎡      \ddot{\x}_{body}      ⎤
⎢                            ⎥
⎢      \ddot{\y}_{body}      ⎥
⎢                            ⎥
⎢    \ddot{\theta}_{body}    ⎥
⎢                            ⎥
⎢     \ddot{\theta}_{UL}     ⎥
⎢                            ⎥
⎢     \ddot{\theta}_{UR}     ⎥
⎢                            ⎥
⎢\ddot{\r\_prismatic}_{footL}⎥
⎢                            ⎥
⎣\ddot{\r\_prismatic}_{footR}⎦

In [7]:
''' Setup positions (to COM) and velocities'''


# define absolute angles
abs_angles = {}
r_links = {}
   
abs_angles.update({('ULA'): th_body + th['UL']})
abs_angles.update({('URA'): th_body + th['UR']})

r_links.update({'body': sym.Matrix([[x_body,
                                     y_body,
                                     th['body']]]).transpose()})

r_links.update({('UL'): sym.Matrix([[x_body-lengths['UL']/2*sym.cos(abs_angles['ULA']),
                                     y_body-lengths['UL']/2*sym.sin(abs_angles['ULA']),
                                     abs_angles['ULA']]]).transpose()})

r_links.update({('footL'): sym.Matrix([[x_body-(lengths['UL']+r_prismatic['footL'])*sym.cos(abs_angles['ULA']),
                                        y_body-(lengths['UL']+r_prismatic['footL'])*sym.sin(abs_angles['ULA']),
                                        0.0]]).transpose()})

r_links.update({('UR'): sym.Matrix([[x_body+lengths['UR']/2*sym.cos(np.pi - abs_angles['URA']),
                                     y_body-lengths['UR']/2*sym.sin(np.pi - abs_angles['URA']),
                                     abs_angles['URA']]]).transpose()})

r_links.update({('footR'): sym.Matrix([[x_body+(lengths['UR']+r_prismatic['footR'])*sym.cos(np.pi - abs_angles['URA']),
                                        y_body-(lengths['UR']+r_prismatic['footR'])*sym.sin(np.pi - abs_angles['URA']),
                                        0.0]]).transpose()})

In [8]:
# the Jacobians
Jbns = {}
dr_links = {}

for link in links:
    Jbns.update({(link): r_links[link].jacobian(q)})
    dr_links.update({(link): Jbns[link]*dq})

In [9]:
Mass_matrix = {}
Ttot = sym.Matrix([[0.0]])
Vtot = 0.0

for link in links:
    Mass_matrix.update({(link): sym.Matrix([[masses[link],masses[link],inertias[link]]])})
    Ttot = Ttot + 0.5*Mass_matrix[link]*sym.matrix_multiply_elementwise(dr_links[link],dr_links[link])
    Vtot = Vtot + masses[link]*g*r_links[link][1]
Ttot = Ttot[0]

In [10]:
# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(Ttot,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([Ttot]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([Vtot]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces
Fp = {}
Tau = {}
GRFs = {}

Fp.update({('left'): sym.symbols('Fp_left')})
Fp.update({('right'): sym.symbols('Fp_right')})

Tau.update({('left'): sym.symbols('tau_left')})
Tau.update({('right'): sym.symbols('tau_right')})

GRFs.update({('x','left'): sym.symbols('GRF_x_left')})
GRFs.update({('y','left'): sym.symbols('GRF_y_left')})
GRFs.update({('x','right'): sym.symbols('GRF_x_right')})
GRFs.update({('y','right'): sym.symbols('GRF_y_right')})

Tau_body = sym.Matrix([[0],[0],[-Tau['left']-Tau['right']]])
Tau_UL = sym.Matrix([[0],[0],[Tau['left']]])
Tau_UR = sym.Matrix([[0],[0],[Tau['right']]])

Fp_UL = sym.Matrix([[-Fp['left']*sym.cos(abs_angles['ULA'])],[Fp['left']*sym.sin(abs_angles['ULA'])],[0]])
Fp_footL = sym.Matrix([[Fp['left']*sym.sin(abs_angles['ULA'])],[-Fp['left']*sym.cos(abs_angles['ULA'])],[0]])

Fp_UR = sym.Matrix([[-Fp['right']*sym.cos(abs_angles['URA'])],[Fp['right']*sym.sin(abs_angles['URA'])],[0]])
Fp_footR = sym.Matrix([[Fp['right']*sym.sin(abs_angles['URA'])],[-Fp['right']*sym.cos(abs_angles['URA'])],[0]])

left_GRF_momentum = -(0.5*lengths['UL']+r_prismatic['footL'])*sym.cos(abs_angles['ULA'])*GRFs['y','left'] + (0.5*lengths['UL']+r_prismatic['footL'])*sym.sin(abs_angles['ULA'])*GRFs['x','left'] 
GRF_footL = sym.Matrix([[GRFs['x','left']],[GRFs['y','left']],[left_GRF_momentum]])

right_GRF_momentum = (0.5*lengths['UR']+r_prismatic['footR'])*sym.cos(abs_angles['URA'])*GRFs['y','right'] + (0.5*lengths['UR']+r_prismatic['footR'])*sym.sin(abs_angles['URA'])*GRFs['x','right'] 
GRF_footR = sym.Matrix([[GRFs['x','right']],[GRFs['y','right']],[right_GRF_momentum]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = Tau_body.transpose()*Jbns['body'][:,j] + (Tau_UL+Fp_UL).transpose()*Jbns['UL'][:,j] + (Tau_UR+Fp_UR).transpose()*Jbns['UR'][:,j] + (Fp_footL+GRF_footL).transpose()*Jbns['footL'][:,j] + (Fp_footR+GRF_footR).transpose()*Jbns['footR'][:,j]


In [14]:
Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = Fp_UL.transpose()*Jbns['UL'][:,j]
Q
    

⎡            -Fp_left⋅cos(θ_UL + θ_body)            ⎤
⎢                                                   ⎥
⎢            Fp_left⋅sin(θ_UL + θ_body)             ⎥
⎢                                                   ⎥
⎢-Fp_left⋅l_UL⋅sin(θ_UL + θ_body)⋅cos(θ_UL + θ_body)⎥
⎢                                                   ⎥
⎢-Fp_left⋅l_UL⋅sin(θ_UL + θ_body)⋅cos(θ_UL + θ_body)⎥
⎢                                                   ⎥
⎢                         0                         ⎥
⎢                                                   ⎥
⎢                         0                         ⎥
⎢                                                   ⎥
⎣                         0                         ⎦

In [11]:
# Manipulator Equation 
EOM = Lg1 - Lg3 + Lg4 - Q

# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('M2_SAVE_EOM_unsimplified.db')

In [12]:
EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

In [13]:
# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('M2_SAVE_EOM_simplified.db')